# First, Evolve 2 populations as normal

### Setup

In [1]:
from deap import base, creator, tools
import random
import string
import itertools

signals = list(string.ascii_lowercase)  # Characters used to make words
meanings = list(range(0, 100))  # 100 possible meanings

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", dict, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def create_word(min_length=1, max_length=3):
    length = random.randint(min_length, max_length)
    return ''.join(random.choice(signals) for _ in range(length))

def create_individual():
    individual = creator.Individual()
    individual.is_bilingual = 0
    individual.local_state = random.choice(meanings)
    individual.age = 0
    for meaning in meanings:

        word = create_word()
        individual[meaning] = word

    return individual

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


### Evaluation

In [2]:
GLOBAL_STATE = random.choice(meanings)

"""
Adds newly heard word to individual's vocabulary 
if a randomly guessed meaning matches that of the 
speaker.
"""
def learn(individual, new_word, speakers_meaning):
    random_meaning = random.choice(meanings)
    if random_meaning == speakers_meaning:
        individual[random_meaning] = new_word

def pairwise_communication(speaker, listener, GLOBAL_STATE):
    fitness = 0
    # speaker encodes signals
    local_meaning = speaker.local_state
    global_meaning = GLOBAL_STATE

    semantic_local = speaker[local_meaning]
    semantic_global = speaker[global_meaning]

    # only update if listener has words in dictionary
    decoded_local = -100
    decoded_global = -101

    localWordPresentinListener = 0
    globalWordPresentinListener = 0
    # listener decodes local signal or possibly learns new word
    for meaning in meanings:
        if semantic_local == listener[meaning]:  
            decoded_local = meaning
            localWordPresentinListener = 1
        if semantic_global == listener[meaning]:
            decoded_global = meaning
            globalWordPresentinListener = 1

    if localWordPresentinListener == 0:
        if (random.random() < 1):
            learn(listener, semantic_local, local_meaning)

    if globalWordPresentinListener == 0:
        if (random.random() < 1): 
            learn(listener, semantic_global, global_meaning)

    if (decoded_local == local_meaning) & (localWordPresentinListener == 1):
        fitness += 1
    if (decoded_global == global_meaning) & (globalWordPresentinListener == 1):
        fitness += 1

    # penalty if different ideas are represented by the same word
    if local_meaning != global_meaning:
        if semantic_global == semantic_local:
            fitness -= 1
        if decoded_local == decoded_global:
            fitness -= 1
        
    return fitness   
            
def survival_task(individual, GROUP_SIZE):
    penalty = 0
    for i in range(2 * (GROUP_SIZE - 1)): 
        meaning1, meaning2 = random.sample(meanings, 2) # two different meanings
        if individual[meaning1] == individual[meaning2]: # same word
            penalty += 2

    return penalty  

# evaluate group
def evaluate_group(group, GROUP_SIZE):
    GLOBAL_STATE = random.choice(meanings)
    # reset fitness
    for ind in group:
        # ind.fitness.values = (0,)  # reset fitness
        ind.local_state = random.choice(meanings)

    # speak
    for speaker, listener in itertools.permutations(group, 2):
        fitness_bonus = pairwise_communication(speaker, listener, GLOBAL_STATE)
        speaker.fitness.values = (speaker.fitness.values[0] + fitness_bonus,)
        listener.fitness.values = (listener.fitness.values[0] + fitness_bonus,)

    # individual survival
    for ind in group:
        penalty = survival_task(ind, GROUP_SIZE)
        ind.fitness.values = (ind.fitness.values[0] - penalty,)


### Declare population + initial eval

In [3]:
# create pop
# baseline: N=100, G=10
population = toolbox.population(n=100)

for ind in population:
    ind.fitness.values = (0,)

toolbox.register("evaluate", evaluate_group)

GROUP_SIZE = 10

# for i in range(500):
groups = [population[i:i + GROUP_SIZE] for i in range(0, len(population), GROUP_SIZE)]
for group in groups:
    toolbox.evaluate(group, GROUP_SIZE)

count = 0
for ind in population:
    if ind.fitness.values[0] > 0:
        print(ind, ind.fitness.values[0])
        count += 1
print(count)

{0: 'uo', 1: 'm', 2: 'e', 3: 'fzk', 4: 'k', 5: 'x', 6: 'rmh', 7: 'lp', 8: 'b', 9: 'joa', 10: 'az', 11: 'k', 12: 'v', 13: 'jb', 14: 'h', 15: 'r', 16: 'krf', 17: 'da', 18: 'vha', 19: 'fz', 20: 'fc', 21: 'f', 22: 'xx', 23: 'y', 24: 'x', 25: 'gia', 26: 'rh', 27: 'wl', 28: 'rt', 29: 'ayb', 30: 'zlo', 31: 'y', 32: 'f', 33: 'jy', 34: 't', 35: 'az', 36: 'mdd', 37: 'b', 38: 'cot', 39: 'ch', 40: 'rn', 41: 'du', 42: 'p', 43: 'wok', 44: 'g', 45: 'lrn', 46: 'wp', 47: 'zjr', 48: 'fz', 49: 'uc', 50: 'jhs', 51: 'g', 52: 'i', 53: 'hqj', 54: 'qk', 55: 'iol', 56: 'l', 57: 'oon', 58: 'jee', 59: 'irn', 60: 'xh', 61: 'd', 62: 'eom', 63: 'k', 64: 'ps', 65: 'ab', 66: 'dpl', 67: 'hs', 68: 'cqr', 69: 'wpx', 70: 'p', 71: 'kzh', 72: 'cz', 73: 'f', 74: 'jz', 75: 'nz', 76: 'gms', 77: 'djq', 78: 'f', 79: 'gl', 80: 'u', 81: 'gs', 82: 'cxy', 83: 'xnz', 84: 'gin', 85: 'd', 86: 'v', 87: 'v', 88: 'y', 89: 'bd', 90: 'hb', 91: 'f', 92: 'yyv', 93: 'vy', 94: 'p', 95: 'g', 96: 'aty', 97: 'yiw', 98: 'a', 99: 'gp'} 1.0
{0: 'crj

### Params

In [4]:
from deap import algorithms
import numpy

# Parameters
NUM_GENERATIONS = 10000
CXPB = 0.5  # Crossover probability (.5)
MUTPB = 0.01  # .01 best (100 meanings))

# mut for signals
def mutateInd(individual, indpb):
    for i in range(len(meanings)):
        if random.random() < indpb:
            individual[i] = create_word()

    return individual,

# tools
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", mutateInd, indpb=0.1)
toolbox.register("select", tools.selRoulette) # inspired by paper
toolbox.register("tourn", tools.selTournament)
toolbox.register("randsel", tools.selRandom)
toolbox.register("stochastic_select", tools.selStochasticUniversalSampling)



stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

# for steady state
def select_next_generation(parents, offspring, max_age=3):
    combined = parents + offspring
    filtered = [ind for ind in combined if ind.age < max_age]
    selected = toolbox.tourn(filtered, len(population), 2)
    return selected

def calculate_average_agreement(population, num_meanings):
    total_agreement = 0
    for i in range(num_meanings):
        gene_values = [individual[i] for individual in population]
        most_common_value, count = Counter(gene_values).most_common(1)[0]
        agreement_percentage = (count / len(population)) * 100
        total_agreement += agreement_percentage
    average_agreement = total_agreement / num_meanings
    return average_agreement




### run algorithm

In [5]:
for gen in range(NUM_GENERATIONS):
    
    #for ES:
    MU = 100
    LAMBDA = 150

    """
    Mating selection
    """
    #offspring = toolbox.randsel(population, LAMBDA)
    #offspring = toolbox.select(population, len(population))
    #offspring = toolbox.tourn(population, len(population), tournsize=2)
    offspring = toolbox.randsel(population, len(population)) # random sel

    offspring = list(map(toolbox.clone, offspring))

    # mutation and crossover
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    """
    Evaluate Fitness
    """
    random.shuffle(offspring)
    for ind in offspring:
        ind.fitness.values = (0,)
    groups = [offspring[i:i + GROUP_SIZE] for i in range(0, len(offspring), GROUP_SIZE)]
    for group in groups:
        toolbox.evaluate(group, GROUP_SIZE)

    for ind in population:
        ind.age += 1

    """
    Survivor Selection
    """
    # ES:
    #population[:] = toolbox.stochastic_select(offspring, MU)

    # steady state:
    population [:] = select_next_generation(population, offspring) 

    # generational:
    #population[:] = offspring


    fits = [ind.fitness.values[0] for ind in population]


    from collections import Counter

    def count_duplicate_values(individual):
        value_count = Counter(individual.values())
        duplicates = [value for value, count in value_count.items() if count > 1]
        return len(duplicates)  # Return the number of unique values that are duplicates

    dup_list = []

    for individual in population:
        num_duplicates = count_duplicate_values(individual)
        dup_list.append(num_duplicates)
       
    sum_dups = 0
    for dup in dup_list:
        sum_dups += dup
    #print("Dup Avg:", sum_dups / len(ind) / len(population))



    length = len(population)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print(f"Generation {gen}: Min {min(fits)}, Max {max(fits)}, Avg {mean}, Std {std}, Avg % of duplicate genes across participants {sum_dups / len(ind) / len(population)}")
    agreement = calculate_average_agreement(population, 100)
    print(calculate_average_agreement(population, 100))
    if (mean == 36.0) & (max(fits) == 36.0) & (min(fits) == 36.0) & (std == 0.0) & (agreement == 100.0) & ((sum_dups / len(ind) / len(population)) == 0.0):
        break
# print final pop
for ind in population:
    print(ind, ind.fitness.values)

Generation 0: Min -2.0, Max 4.0, Avg 0.34, Std 0.9079647570252933, Avg % of duplicate genes across participants 0.1042
7.13
Generation 1: Min -2.0, Max 14.0, Avg 1.48, Std 2.9647259569815216, Avg % of duplicate genes across participants 0.09849999999999999
10.42
Generation 2: Min 0.0, Max 14.0, Avg 3.83, Std 3.932060528526996, Avg % of duplicate genes across participants 0.09849999999999999
15.62
Generation 3: Min 0.0, Max 12.0, Avg 3.25, Std 2.812027738127773, Avg % of duplicate genes across participants 0.09910000000000001
23.19
Generation 4: Min 0.0, Max 9.0, Avg 4.51, Std 2.1931484217900072, Avg % of duplicate genes across participants 0.1014
28.82
Generation 5: Min 1.0, Max 13.0, Avg 6.41, Std 3.0003166499554674, Avg % of duplicate genes across participants 0.0955
31.49
Generation 6: Min 1.0, Max 17.0, Avg 7.36, Std 3.793468070249175, Avg % of duplicate genes across participants 0.09380000000000001
33.38
Generation 7: Min 1.0, Max 18.0, Avg 10.7, Std 4.323193264243461, Avg % of du

KeyboardInterrupt: 

# Bilingualism

In [ ]:
# if agreement is 100%, all individuals should be identical.
for ind in population:
    print(ind)

### save languages

In [6]:
population1 = toolbox.population(n=100)
population2 = toolbox.population(n=100)


# predefined languages were copy and pasted from an individual in a perfect population. Since the population was 100% agreement, we simply rewrite using this genome:
predefined_language = {0: 'bi', 1: 'tgg', 2: 'a', 3: 'v', 4: 've', 5: 'ka', 6: 'i', 7: 'fm', 8: 'gw', 9: 'ha', 10: 'eol', 11: 'rqi', 12: 'r', 13: 'zxw', 14: 'ab', 15: 'fnl', 16: 'mju', 17: 'w', 18: 'nhm', 19: 'qs', 20: 'dgl', 21: 'n', 22: 'zou', 23: 'fo', 24: 'mmx', 25: 'u', 26: 'zh', 27: 'qi', 28: 'lrv', 29: 'vtg', 30: 'bjj', 31: 'vw', 32: 'hgk', 33: 'ttm', 34: 'vo', 35: 'fk', 36: 'up', 37: 'end', 38: 'fd', 39: 'zwl', 40: 'sm', 41: 'b', 42: 'wt', 43: 'qtj', 44: 'z', 45: 'yl', 46: 'zms', 47: 'fos', 48: 'kp', 49: 'y', 50: 'akh', 51: 'j', 52: 'hcd', 53: 'ed', 54: 'mw', 55: 'ow', 56: 'rl', 57: 'xp', 58: 'xqx', 59: 'yy', 60: 'lll', 61: 'ys', 62: 'ifh', 63: 'p', 64: 'g', 65: 'ob', 66: 'ur', 67: 'k', 68: 'acz', 69: 'kwt', 70: 'qt', 71: 'ax', 72: 'pn', 73: 'ly', 74: 'oxu', 75: 'ru', 76: 'c', 77: 'les', 78: 'vso', 79: 'pjy', 80: 'qpm', 81: 'scr', 82: 'hdh', 83: 'm', 84: 'gnj', 85: 'ut', 86: 'bu', 87: 'uu', 88: 'yf', 89: 's', 90: 'jdy', 91: 'wte', 92: 'ufo', 93: 'yw', 94: 'zd', 95: 'l', 96: 'd', 97: 'na', 98: 'iik', 99: 'qmj'}
for ind in population1:
    for meaning in meanings:
        ind[meaning] = predefined_language[meaning]

predefined_language2 = {0: 'xg', 1: 'os', 2: 'mfz', 3: 'it', 4: 'yy', 5: 'bzd', 6: 'nh', 7: 'ihg', 8: 'azu', 9: 'uul', 10: 'wem', 11: 'qwc', 12: 'ik', 13: 'dfb', 14: 'sdl', 15: 'hn', 16: 'vo', 17: 'ug', 18: 'vte', 19: 'iw', 20: 'sf', 21: 'vt', 22: 'g', 23: 'b', 24: 'pa', 25: 'god', 26: 'vf', 27: 'y', 28: 'ud', 29: 'db', 30: 'ax', 31: 'n', 32: 'in', 33: 'ztu', 34: 'syu', 35: 'd', 36: 'ic', 37: 'qem', 38: 'rux', 39: 'wt', 40: 'hp', 41: 'sh', 42: 'ak', 43: 'dsh', 44: 'bur', 45: 'jt', 46: 'j', 47: 'cqw', 48: 'tdk', 49: 'sy', 50: 'z', 51: 'm', 52: 'kvm', 53: 'old', 54: 'bqz', 55: 'crw', 56: 'eys', 57: 'h', 58: 'vh', 59: 'bk', 60: 'fzo', 61: 'u', 62: 'jdu', 63: 's', 64: 'npx', 65: 'xs', 66: 'ob', 67: 'i', 68: 'asx', 69: 'tdn', 70: 'th', 71: 'k', 72: 'vrs', 73: 'ozq', 74: 'bp', 75: 'ih', 76: 'rm', 77: 'no', 78: 'sr', 79: 'qyk', 80: 'yqq', 81: 'em', 82: 'xb', 83: 'tq', 84: 'c', 85: 'ox', 86: 'geu', 87: 'yu', 88: 'yc', 89: 'td', 90: 'qa', 91: 'r', 92: 'dd', 93: 'e', 94: 'od', 95: 'xj', 96: 'am', 97: 'wnw', 98: 'tl', 99: 'buf'}

for ind in population2:
    for meaning in meanings:
        ind[meaning] = predefined_language2[meaning]



### display

In [7]:
print("POP 1")
for ind in population1:
    print(ind)
print("POP 2")
for ind in population2:
    print(ind)

POP 1
{0: 'bi', 1: 'tgg', 2: 'a', 3: 'v', 4: 've', 5: 'ka', 6: 'i', 7: 'fm', 8: 'gw', 9: 'ha', 10: 'eol', 11: 'rqi', 12: 'r', 13: 'zxw', 14: 'ab', 15: 'fnl', 16: 'mju', 17: 'w', 18: 'nhm', 19: 'qs', 20: 'dgl', 21: 'n', 22: 'zou', 23: 'fo', 24: 'mmx', 25: 'u', 26: 'zh', 27: 'qi', 28: 'lrv', 29: 'vtg', 30: 'bjj', 31: 'vw', 32: 'hgk', 33: 'ttm', 34: 'vo', 35: 'fk', 36: 'up', 37: 'end', 38: 'fd', 39: 'zwl', 40: 'sm', 41: 'b', 42: 'wt', 43: 'qtj', 44: 'z', 45: 'yl', 46: 'zms', 47: 'fos', 48: 'kp', 49: 'y', 50: 'akh', 51: 'j', 52: 'hcd', 53: 'ed', 54: 'mw', 55: 'ow', 56: 'rl', 57: 'xp', 58: 'xqx', 59: 'yy', 60: 'lll', 61: 'ys', 62: 'ifh', 63: 'p', 64: 'g', 65: 'ob', 66: 'ur', 67: 'k', 68: 'acz', 69: 'kwt', 70: 'qt', 71: 'ax', 72: 'pn', 73: 'ly', 74: 'oxu', 75: 'ru', 76: 'c', 77: 'les', 78: 'vso', 79: 'pjy', 80: 'qpm', 81: 'scr', 82: 'hdh', 83: 'm', 84: 'gnj', 85: 'ut', 86: 'bu', 87: 'uu', 88: 'yf', 89: 's', 90: 'jdy', 91: 'wte', 92: 'ufo', 93: 'yw', 94: 'zd', 95: 'l', 96: 'd', 97: 'na', 98: 

In [8]:
from deap import base, creator, tools
import random
import string
import itertools

toolbox = base.Toolbox()

def create_bilingual_individual():
    individual = creator.Individual()
    individual.local_state = random.choice(meanings)
    individual.is_bilingual = 1
    # capacity for two languages
    individual.language1 = {}
    individual.language2 = {}
    individual.age = 0
    for meaning in meanings:
        word = create_word()
        individual.language1[meaning] = word
        word = create_word()
        individual.language2[meaning] = word

    return individual

toolbox.register("bilingual_individual", create_bilingual_individual)
toolbox.register("bilingual_population", tools.initRepeat, list, toolbox.bilingual_individual)


## Redefine communication methods to work with bilingual individuals

In [9]:
GLOBAL_STATE = random.choice(meanings)

"""
Adds newly heard word to individual's vocabulary 
if a randomly guessed meaning matches that of the 
speaker.
"""
def learn(individual, new_word, speakers_meaning, otherWordSpoken):
    random_meaning = random.choice(meanings)
    useLang1 = 0
    useLang2 = 0
    if random_meaning == speakers_meaning:
        for meaning in meanings:
            if individual.language1[meaning] == otherWordSpoken: # if other word belongs to lang 1
                useLang1 = 1
            elif individual.language2[meaning] == otherWordSpoken: # if other word belongs to lang 2
                useLang2 = 1
        if useLang1 == 1:
            individual.language1[random_meaning] = new_word
        elif useLang2 == 1:
            individual.language1[random_meaning] = new_word
        else: # if other word hasn't been heard
            if random.random() < .5:
                individual.language1[random_meaning] = new_word
            else:
                individual.language2[random_meaning] = new_word

def pairwise_communication(speaker, listener, GLOBAL_STATE):
    fitness = 0
    # speaker encodes signals
    local_meaning = speaker.local_state
    global_meaning = GLOBAL_STATE

    if speaker.is_bilingual == 1:
        if random.random() < .5: # use lang 1
            semantic_local = speaker.language1[local_meaning]
            semantic_global = speaker.language1[global_meaning]
        else: # use lang 2
            semantic_local = speaker.language2[local_meaning]
            semantic_global = speaker.language2[global_meaning]
    else:
        semantic_local = speaker[local_meaning]
        semantic_global = speaker[global_meaning]

    # only update if listener has words in dictionary
    decoded_local = -100
    decoded_global = -101

    localWordPresentinListener = 0
    globalWordPresentinListener = 0
    # listener decodes local signal or possibly learns new word
    for meaning in meanings:
        if listener.is_bilingual == 1:
            if semantic_local == listener.language1[meaning]:
                decoded_local = meaning
                localWordPresentinListener = 1
            elif semantic_local == listener.language2[meaning]:
                decoded_local = meaning
                localWordPresentinListener = 1
            if semantic_global == listener.language1[meaning]:
                decoded_global = meaning
                localWordPresentinListener = 1
            elif semantic_global == listener.language2[meaning]:
                decoded_global = meaning
                localWordPresentinListener = 1

        else:
            if semantic_local == listener[meaning]:  
                decoded_local = meaning
                localWordPresentinListener = 1
            if semantic_global == listener[meaning]:
                decoded_global = meaning
                globalWordPresentinListener = 1

    if localWordPresentinListener == 0:
        if (random.random() < 1):
            if listener.is_bilingual == 1:
                learn(listener, semantic_local, local_meaning, semantic_global)

    if globalWordPresentinListener == 0:
        if (random.random() < 1): 
            if listener.is_bilingual == 1:
                learn(listener, semantic_global, global_meaning, semantic_local)

    if (decoded_local == local_meaning) & (localWordPresentinListener == 1):
        fitness += 1
    if (decoded_global == global_meaning) & (globalWordPresentinListener == 1):
        fitness += 1

    # penalty if different ideas are represented by the same word
    if local_meaning != global_meaning:
        if semantic_global == semantic_local:
            fitness -= 1
        if decoded_local == decoded_global:
            fitness -= 1
        
    return fitness   
            
def survival_task(individual, GROUP_SIZE):
    penalty = 0
    for i in range(2 * (GROUP_SIZE - 1)): 
        meaning1, meaning2 = random.sample(meanings, 2) # two different meanings
        if individual.language1[meaning1] == individual.language1[meaning2]: # same word
            penalty += 2
        if individual.language2[meaning1] == individual.language2[meaning2]: # same word
            penalty += 2

    return penalty  

# evaluate group
def evaluate_group(bilingual_group, group1, group2, GROUP_SIZE):
    GLOBAL_STATE = random.choice(meanings)
    collective = bilingual_group + group1 + group2

    for ind in collective:
        # ind.fitness.values = (0,)  # reset fitness
        ind.local_state = random.choice(meanings)

    # speak with different languages

    for speaker, listener in itertools.permutations(collective, 2):
        fitness_bonus = pairwise_communication(speaker, listener, GLOBAL_STATE)
        if speaker.is_bilingual == 1:
            speaker.fitness.values = (speaker.fitness.values[0] + fitness_bonus,)
        if listener.is_bilingual == 1:
            listener.fitness.values = (listener.fitness.values[0] + fitness_bonus,)

    # individual survival
    for ind in bilingual_group:
        penalty = survival_task(ind, GROUP_SIZE)
        ind.fitness.values = (ind.fitness.values[0] - penalty,)


# initialize population

In [20]:
# create pop
# baseline: N=100, G=10
bilingual_population = toolbox.bilingual_population(n=100)

for ind in bilingual_population:
    ind.fitness.values = (0,)

toolbox.register("evaluate", evaluate_group)

GROUP_SIZE = 10

# for i in range(500):
bilingual_groups = [bilingual_population[i:i + GROUP_SIZE] for i in range(0, len(bilingual_population), GROUP_SIZE)]
for bilingual_group in bilingual_groups:
    group1 = random.sample(population1, GROUP_SIZE)
    group2 = random.sample(population2, GROUP_SIZE)
    toolbox.evaluate(bilingual_group, group1, group2, GROUP_SIZE)

count = 0
for ind in bilingual_population:
    if ind.fitness.values[0] > 0:
        print(ind.language1, ind.fitness.values[0])
        print(ind.language2, ind.fitness.values[0])
        count += 1
print(count)

{0: 'ia', 1: 'yi', 2: 'q', 3: 'ab', 4: 'j', 5: 'fu', 6: 'rj', 7: 'ie', 8: 'md', 9: 'y', 10: 'ufi', 11: 'frb', 12: 'eae', 13: 'qs', 14: 'v', 15: 'vmy', 16: 'p', 17: 'bje', 18: 'wr', 19: 'wx', 20: 'w', 21: 'o', 22: 'hs', 23: 'vm', 24: 'ss', 25: 'tu', 26: 'f', 27: 'i', 28: 'use', 29: 'jb', 30: 'vsw', 31: 'kg', 32: 'm', 33: 'tbs', 34: 'pib', 35: 'h', 36: 'jeo', 37: 'uqi', 38: 'g', 39: 'bo', 40: 'zfp', 41: 'q', 42: 'y', 43: 'm', 44: 'lw', 45: 'htm', 46: 'qzi', 47: 'l', 48: 'g', 49: 'jlw', 50: 'lcs', 51: 'o', 52: 'fgv', 53: 'rtz', 54: 'pt', 55: 'wn', 56: 'cxx', 57: 'qvv', 58: 'bhj', 59: 'k', 60: 'jk', 61: 'yrg', 62: 'tt', 63: 'e', 64: 'm', 65: 'tpv', 66: 'szo', 67: 'uv', 68: 'qal', 69: 'k', 70: 'zr', 71: 'kj', 72: 'ton', 73: 'sb', 74: 'ku', 75: 'xf', 76: 'e', 77: 'p', 78: 'sk', 79: 'avy', 80: 'e', 81: 'j', 82: 'w', 83: 'xn', 84: 'k', 85: 'hu', 86: 'ub', 87: 't', 88: 'ry', 89: 'gn', 90: 'qzz', 91: 'x', 92: 'l', 93: 'aoq', 94: 'x', 95: 'htx', 96: 'dvu', 97: 'gc', 98: 'nr', 99: 'wzk'} 6.0
{0: '

### params

In [21]:
from deap import algorithms
import numpy

# Parameters
NUM_GENERATIONS = 10000
CXPB = 0.5  # Crossover probability (.5)
MUTPB = 0.03  # .01 best (100 meanings))

# mut for signals
def mutateInd(individual, indpb):
    for i in range(len(meanings)):
        if random.random() < .01:
            individual.language1[i] = create_word()
        if random.random() < .01:
            individual.language2[i] = create_word()
        if random.random() < .01: # swap
            temp = individual.language1[i]
            individual.language1[i] = individual.language2[i]
            individual.language2[i] = temp
    return individual,

# tools
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", mutateInd, indpb=0.1)
toolbox.register("select", tools.selRoulette) # inspired by paper
toolbox.register("tourn", tools.selTournament)
toolbox.register("randsel", tools.selRandom)
toolbox.register("stochastic_select", tools.selStochasticUniversalSampling)



stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

# for steady state
def select_next_generation(parents, offspring, max_age=3):
    combined = parents + offspring
    filtered = [ind for ind in combined if ind.age < max_age]
    selected = toolbox.tourn(filtered, len(population), 2)
    return selected

def calculate_average_agreement(population, num_meanings):
    total_agreement = 0
    # lang 1
    for i in range(num_meanings):
        gene_values = [individual.language1[i] for individual in population]
        most_common_value, count = Counter(gene_values).most_common(1)[0]
        agreement_percentage = (count / len(population)) * 100
        total_agreement += agreement_percentage
    average_agreement1 = total_agreement / num_meanings

    total_agreement = 0
    for i in range(num_meanings):
        gene_values = [individual.language2[i] for individual in population]
        most_common_value, count = Counter(gene_values).most_common(1)[0]
        agreement_percentage = (count / len(population)) * 100
        total_agreement += agreement_percentage
    average_agreement2 = total_agreement / num_meanings


    return average_agreement1, average_agreement2 




### run

In [22]:
for gen in range(NUM_GENERATIONS):
    
    #for ES:
    MU = 100
    LAMBDA = 150

    """
    Mating selection
    """
    #offspring = toolbox.randsel(population, LAMBDA)
    #offspring = toolbox.select(population, len(population))
    #offspring = toolbox.tourn(population, len(population), tournsize=2)
    offspring = toolbox.randsel(bilingual_population, len(bilingual_population)) # random sel

    offspring = list(map(toolbox.clone, offspring))

    # mutation and crossover
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1.language1, child2.language1)
            toolbox.mate(child1.language2, child2.language2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    """
    Evaluate Fitness
    """
    random.shuffle(offspring)
    for ind in offspring:
        ind.fitness.values = (0,)
    bilingual_groups = [offspring[i:i + GROUP_SIZE] for i in range(0, len(offspring), GROUP_SIZE)]
    for bilingual_group in bilingual_groups:
        group1 = random.sample(population1, GROUP_SIZE)
        group2 = random.sample(population2, GROUP_SIZE)
        toolbox.evaluate(bilingual_group, population1, population2, GROUP_SIZE)

    for ind in bilingual_population:
        ind.age += 1

    """
    Survivor Selection
    """
    # ES:
    #population[:] = toolbox.stochastic_select(offspring, MU)

    # steady state:
    bilingual_population [:] = select_next_generation(bilingual_population, offspring) 

    # generational:
    #population[:] = offspring


    fits = [ind.fitness.values[0] for ind in bilingual_population]


    from collections import Counter

    def count_duplicate_values(individual):
        value_count = Counter(individual.values())
        duplicates = [value for value, count in value_count.items() if count > 1]
        return len(duplicates)  # Return the number of unique values that are duplicates

    dup_list = []

    for individual in bilingual_population:
        num_duplicates = count_duplicate_values(individual.language1) + count_duplicate_values(individual.language2)
        dup_list.append(num_duplicates)
       
    sum_dups = 0
    for dup in dup_list:
        sum_dups += dup
    #print("Dup Avg:", sum_dups / len(ind) / len(population))



    length = len(bilingual_population)
    print("len", length)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print(f"Generation {gen}: Min {min(fits)}, Max {max(fits)}, Avg {mean}, Std {std}, Avg % of duplicate genes across participants {sum_dups / len(ind.language1) / len(bilingual_population)}")
    agreement = calculate_average_agreement(bilingual_population, 100)
    print(calculate_average_agreement(bilingual_population, 100))
    if (mean == 36.0) & (max(fits) == 36.0) & (min(fits) == 36.0) & (std == 0.0) & (agreement == 100.0) & ((sum_dups / len(ind.language1) / len(bilingual_population)) == 0.0):
        break
# print final pop
for ind in bilingual_population:
    print(ind, ind.fitness.values)

len 100
Generation 0: Min -1.0, Max 44.0, Avg 2.2, Std 4.679743582719036, Avg % of duplicate genes across participants 0.20199999999999999
(7.08, 7.0)
len 100
Generation 1: Min 0.0, Max 65.0, Avg 8.65, Std 15.57201014641334, Avg % of duplicate genes across participants 0.21
(11.69, 11.65)
len 100
Generation 2: Min 1.0, Max 65.0, Avg 11.69, Std 16.538255651670163, Avg % of duplicate genes across participants 0.20629999999999998
(19.25, 19.02)
len 100
Generation 3: Min 4.0, Max 106.0, Avg 20.42, Std 23.22850834642638, Avg % of duplicate genes across participants 0.1999
(22.41, 22.64)
len 100
Generation 4: Min 4.0, Max 110.0, Avg 26.55, Std 26.201288136272996, Avg % of duplicate genes across participants 0.21030000000000001
(25.0, 24.59)
len 100
Generation 5: Min 6.0, Max 118.0, Avg 36.65, Std 32.98435235077385, Avg % of duplicate genes across participants 0.2069
(25.35, 24.89)
len 100
Generation 6: Min 8.0, Max 111.0, Avg 36.15, Std 30.01045651102295, Avg % of duplicate genes across part

KeyboardInterrupt: 

### check 

In [23]:
for ind in bilingual_population:
    print("Language 1:")
    print(ind.language1)
    print("Language 2:")
    print(ind.language2)

print("Lang 1")
print(predefined_language)
print("Lang 2")
print(predefined_language2)

Language 1:
{0: 'xg', 1: 'tgg', 2: 'mfz', 3: 'v', 4: 'yy', 5: 'bzd', 6: 'i', 7: 'ihg', 8: 'azu', 9: 'uul', 10: 'wem', 11: 'qwc', 12: 'ls', 13: 'dfb', 14: 'sdl', 15: 'hn', 16: 'mju', 17: 'ug', 18: 'nhm', 19: 'qs', 20: 'dgl', 21: 'vt', 22: 'g', 23: 'b', 24: 'mmx', 25: 'bwe', 26: 'vf', 27: 'qi', 28: 'lrv', 29: 'db', 30: 'ax', 31: 'n', 32: 'in', 33: 'ttm', 34: 'syu', 35: 'd', 36: 'up', 37: 'qw', 38: 'rux', 39: 'zwl', 40: 'sm', 41: 'b', 42: 'ak', 43: 'jtk', 44: 'bur', 45: 'jt', 46: 'lc', 47: 'cqw', 48: 'tdk', 49: 'sy', 50: 'z', 51: 'm', 52: 'kvm', 53: 'c', 54: 'bqz', 55: 'crw', 56: 'eys', 57: 'h', 58: 'xqx', 59: 'yy', 60: 'fzo', 61: 'u', 62: 'jdu', 63: 's', 64: 'npx', 65: 'xs', 66: 'ob', 67: 'i', 68: 'acz', 69: 'kwt', 70: 'th', 71: 'k', 72: 'pn', 73: 'ozq', 74: 'oxu', 75: 'ru', 76: 'rm', 77: 'no', 78: 'sr', 79: 'qyk', 80: 'yqq', 81: 'scr', 82: 'xb', 83: 'tq', 84: 'c', 85: 'ox', 86: 'bu', 87: 'uu', 88: 'yc', 89: 'n', 90: 'jdy', 91: 'g', 92: 'dd', 93: 'yw', 94: 'od', 95: 'xj', 96: 'am', 97: '